In [1]:
import tifffile
python_test = __import__('0_python_test')
import pandas as pd
import napari
import numpy as np
import skimage
from scipy import ndimage
from cellpose import models
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
raw_path = 'D:\\Bestun\\raw_data\\ModelTest\\'
#images = os.listdir(raw_path)
model_path = 'D:\\Bestun\\modelTest\\cellpose_train\\models\\cellpose_residual_on_style_on_concatenation_off_cellpose_train_2025_02_06_16_22_53.615012'
save_path = 'D:\\Bestun\\masks_tracked\\ModelTest\\'

images = ['231224_cluster_1h_2.tif']

occurrence_limit = 80   # Percentage of images a cell has to be in order to be accounted for

#dist_limit: the longest distance, in pixels, that the center of mass is allowed
#to move from one image to the next for it to still count as the same
#cell. Make bigger if cells are moving a lot.

#backtrack_limit: the maximum number of images back that the algorithm will search
#through to find a center of mass within the distance limit
#(dist_limit). Make smaller if cells are moving a lot. Can be adjusted in the get_tracked_masks function in 0_python_test.py

#TODO: evaluate the effect of different settings for diameter and dist_limit 
#TODO: evaluate the effect of changing flow_threshold
#TODO: continue looking at the tracking function to make it avoid putting the same label on several cells

T = 5 # Period between images (seconds)
# If generating new masks
diam = 30   # standard setting when trained
dist_limit = diam / 2    # Slightly less than half average diameter
#backtrack_limit = 15

In [3]:
def track_Y(X, model, diam=diam):
    
    #Original flow_threshold = 0.4
    
    # remove empty images
    mean = np.mean(X, axis=(1,2))

    crop = list(np.where(mean == 0))
    if len(crop[0])>0:
        X=X[:crop[0][0],:,:]
    else:
        X = X

    X = [np.squeeze(i) for i in np.split(X, X.shape[0])]
    # predict labels on model
    Y = []
    for x in tqdm(X, desc = 'Segmenting timeseries', unit = 't'):
        
        y, flows, styles = model.eval(x,
                                   diameter=diam, channels=[0,0],
                                      flow_threshold=0.4, do_3D=False)
        Y.append(y)
    print('Track masks over time')
    Y = python_test.get_tracked_masks(masks = np.array(Y))

    return Y

In [ ]:
for i in range(len(images)):
    print('Image name: ' + images[i])
    if images[i] in os.listdir(save_path):
        print('Segmentation for ' + images[i] + ' already exists.')
        continue
    else:
        X = tifffile.imread(raw_path + images[i])
        X = X[:,:,:]
        # load model
        model = models.CellposeModel(gpu = True, pretrained_model=model_path)
        Y = track_Y(X, model)
        tifffile.imwrite(save_path + images[i], Y, imagej=True, metadata={'axes':'TYX'})
        print('Masks saved.')

        print('Masks processing complete for', images[i])

Image name: 231224_cluster_1h_2.tif


Segmenting timeseries: 100%|██████████| 721/721 [03:51<00:00,  3.11t/s]

Track masks over time
Input masks shape: (721, 512, 512)
Unique labels in the first frame: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61]
